## Computer architecures classification :

1. SISD stands for Single Instruction, Single Data. It's a computer architecture where a single processor executes one instruction at a time on a single data stream. This means that instructions are processed sequentially, one after the other, making SISD a sequential processing system.

2. SIMD stands for Single Instruction, Multiple Data. It's a parallel processing technique where a single instruction is applied to multiple data elements simultaneously, typically using special wide registers.

3. MISD, which stands for Multiple Instruction, Single Data, is a type of parallel computer architecture where multiple processing units operate on the same data stream using different instructions. It's primarily a theoretical concept, and no real-world systems are built using it

4. MIMD stands for Multiple Instruction, Multiple Data. It's a computer architecture where multiple processors can execute different instructions on different data simultaneously.


CUDA follows different architecure known as SIMD.
SIMD stands for single instruction multiple threads.

SIMD stands for Single Instruction, Multiple Data. It's a parallel processing technique where a single instruction is applied to multiple data elements simultaneously, typically using special wide registers.

In CUDA
- thread blocks is going to execute in single SM. Multiple thread block can be execute simultaneously on same SM depending on resoucre limitation on SM
- But one thread block cannot be executing in multiple SM. If device cannot run single block in one SM, then error will return for that kernel launch.

SM : In CUDA, SM stands for Streaming Multiprocessor, a fundamental processing unit on NVIDIA GPUs. It's essentially a general-purpose processor that executes CUDA kernels in parallel, responsible for processing data through the GPU.


The equivalent of software to hardware is

|| Thread --- CUDA Core ||

|| Thread block --- SM ||

|| Grid --- Device ||

## Warps

From hardware point of view all threads cannot run in parallel. In software we assume that, but hardware has limitations.

One block runs in one streamming Multiprocessor (SM).

- thread blocks are divide in to smaller units called warps each having 32 consecutive threads.

No of warps per block = Block size / Warp size

Warps are basic unit of execution in a SM.

Once a thread block is scheduled to an SM, threads in the thread block are further partitioned into warps.

And all threads in a warp are executed in Single Instruction Multiple thread fashion.

Each thread has a unique ID.

Warp size is 32 and even to run thread block with single thread, still CUDA runtime will assign single warp which means 32 threads. In this case only 1 thread will be active and other 31 threads will be in inactive state.

resoucre allocations like shared memory for block will be done considering number of warps.

having inactive threads in warp will be a greate waste of resoucre in SM.

There are no any build varaible to indiicate the warp index. But we can get it bby divind threadId.x value by warp size, 32.


In [ ]:
%%writefile hello.cu
#include <cstdio>

__global__ void print_details_of_warps() {

  int gid = blockIdx.y * gridDim.x * blockDim.x + blockDim.x + threadIdx.x ;
  int warp_id = threadIdx.x / 32;
  int gbid = blockIdx.y * gridDim.x + blockIdx.x ;

  printf("tid : %d, bid.x : %d , bid.y : %d, gid: %d, warp_id: %d, gbid: %d \n",
        threadIdx.x, blockIdx.x, blockIdx.y, gid, warp_id, gbid);
}

int main() {
    dim3 block_size(42);
    dim3 grid_size(2, 2);

    print_details_of_warps <<<grid_size, block_size>>> ();
    // Synchronize the device to make sure the output is flushed
    cudaDeviceSynchronize();

    return 0;
}


Writing hello.cu


In [ ]:
!nvcc hello.cu -o hello -gencode arch=compute_75,code=sm_75
!./hello

tid : 32, bid.x : 0 , bid.y : 1, gid: 158, warp_id: 1, gbid: 2 
tid : 33, bid.x : 0 , bid.y : 1, gid: 159, warp_id: 1, gbid: 2 
tid : 34, bid.x : 0 , bid.y : 1, gid: 160, warp_id: 1, gbid: 2 
tid : 35, bid.x : 0 , bid.y : 1, gid: 161, warp_id: 1, gbid: 2 
tid : 36, bid.x : 0 , bid.y : 1, gid: 162, warp_id: 1, gbid: 2 
tid : 37, bid.x : 0 , bid.y : 1, gid: 163, warp_id: 1, gbid: 2 
tid : 38, bid.x : 0 , bid.y : 1, gid: 164, warp_id: 1, gbid: 2 
tid : 39, bid.x : 0 , bid.y : 1, gid: 165, warp_id: 1, gbid: 2 
tid : 40, bid.x : 0 , bid.y : 1, gid: 166, warp_id: 1, gbid: 2 
tid : 41, bid.x : 0 , bid.y : 1, gid: 167, warp_id: 1, gbid: 2 
tid : 32, bid.x : 0 , bid.y : 0, gid: 74, warp_id: 1, gbid: 0 
tid : 33, bid.x : 0 , bid.y : 0, gid: 75, warp_id: 1, gbid: 0 
tid : 34, bid.x : 0 , bid.y : 0, gid: 76, warp_id: 1, gbid: 0 
tid : 35, bid.x : 0 , bid.y : 0, gid: 77, warp_id: 1, gbid: 0 
tid : 36, bid.x : 0 , bid.y : 0, gid: 78, warp_id: 1, gbid: 0 
tid : 37, bid.x : 0 , bid.y : 0, gid: 79, war

## Warp Divergence

Warp divergence occurs when threads within the same warp follow different execution paths through a conditional branch. Since all threads in a warp must execute the same instruction at any given cycle, divergent paths are executed serially—one path at a time—masking out threads not taking that path.

- Warp is diverge when there is multiple path of execution with in same warp. So condition checks, which result in all thread executing same path, will not induce any wrap divergence.

if (tid % 2 != 0)
{
  //do something
}
else
{
  // do something else
}


In the above statment there are 2 branches.

$Branch Efficiency = 100 * ( #Branches - #Divergent_Branches  / #Branches)$

$= 100 * (2-1) / 2 = 50%$

In [ ]:
%%writefile hello.cu
#include <cstdio>

__global__ void code_without_divergence() {

  int gid = blockIdx.x * blockDim.x + threadIdx.x ;
  float a, b;
  a=b=0;
  int warp_id = gid / 32;
  if (warp_id%2 ==0){
    a = 100.0;
    b= 50.0;
  }
  else{
    a = 200;
    b = 75;
  }
  // Add dummy computation
  for (int i = 0; i < 1000; ++i) {
        a = a * 1.00001f + b * 0.99999f;
    }
}

__global__ void divergence_code() { // consecutive threads will take difference part of executions

  int gid = blockIdx.x * blockDim.x + threadIdx.x ;
  float a, b;
  a=b=0;
  if (gid%2 ==0){
    a = 100.0;
    b= 50.0;
  }
  else{
    a = 200;
    b = 75;
  }
  // Add dummy computation
  for (int i = 0; i < 1000; ++i) {
        a = a * 1.00001f + b * 0.99999f;
    }
}

int main() {
  printf("\n-----------------------WARP DIVERGENCE EXAMPLE-------------------------------\n\n");
  int size = 1<<22;
  dim3 block_size(128);
  dim3 grid_size((size + block_size.x - 1) / block_size.x);

  code_without_divergence<<<grid_size,block_size>>>();
  cudaDeviceSynchronize();


  divergence_code<<<grid_size,block_size>>>();
  // Synchronize the device to make sure the output is flushed
  cudaDeviceSynchronize();

    return 0;
}


Overwriting hello.cu


In [ ]:
!nvcc hello.cu -o hello.out -gencode arch=compute_75,code=sm_75
!./hello.out   # To just run the program



-----------------------WARP DIVERGENCE EXAMPLE-------------------------------



In [ ]:
!nvcc hello.cu -o hello.out -gencode arch=compute_75,code=sm_75
!ncu --metrics sm__branch_efficiency.avg.pct ./hello.out



-----------------------WARP DIVERGENCE EXAMPLE-------------------------------

==PROF== Connected to process 473 (/content/hello.out)
==PROF== Profiling "code_without_divergence()" - 0: 0%....50%....100% - 1 pass
==PROF== Profiling "divergence_code()" - 1: 0%....50%....100% - 1 pass
==PROF== Disconnected from process 473
[473] hello.out@127.0.0.1
  code_without_divergence() (32768, 1, 1)x(128, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: Command line profiler metrics
    ----------------------------- ----------- ------------
    Metric Name                   Metric Unit Metric Value
    ----------------------------- ----------- ------------
    sm__branch_efficiency.avg.pct                  (!) n/a
    ----------------------------- ----------- ------------

  divergence_code() (32768, 1, 1)x(128, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: Command line profiler metrics
    ----------------------------- ----------- ------------
    Metric Name                

In [ ]:
!ncu --set full ./hello.out
## full wrap divergence details are shown below:


-----------------------WARP DIVERGENCE EXAMPLE-------------------------------

==PROF== Connected to process 513 (/content/hello.out)
==PROF== Profiling "code_without_divergence()" - 0: 0%....50%....100% - 30 passes
==PROF== Profiling "divergence_code()" - 1: 0%....50%....100% - 30 passes
==PROF== Disconnected from process 513
[513] hello.out@127.0.0.1
  code_without_divergence() (32768, 1, 1)x(128, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         4.96
    SM Frequency                    Mhz       584.58
    Elapsed Cycles                cycle       38,819
    Memory Throughput                 %         0.07
    DRAM Throughput                   %         0.01
    Duration                         us        66.40
    L1/TEX Cache Throughput    

## Resource Partioning and latency hiding

The local execution context of a warp mainly consists of the following resources:
- Program counters
- Registers
- Shared memory

The execution context of each warp processed by a SM is maintained on-chip during the entire lifteime of the warp. Therefore, swtiching from one execution context to another has no cost.

Registers and shared memory can be directly controlled by the programmer.

set of 32-bit registers stored in a register file that are partitioned among threads, and a fixed amount of shared memory that is partitioned among thread blocks.

Fewer warps with more register per thread -- more warps with fewere register per thread

Fewer blocks with more shared memory per block -- more blocks with less shared memory per block

Warp categories in SM

- Active blocks / warps -- resources have been allocated
- Selected warp -- actively executing
- Stalled warp -- not ready for execution
- Eligible warp -- ready for execution but not currently executing

Eligbble for warps

- 32 cuda cores should free for execution

- all arguments to the current instruction for that warp to be ready

What is latency?

 >number of clock cycles between instruction being issued and being completed

- Arithmetic instruction latency
- Memory operation latency


Latency Hiding:

The exeuction context of each warp processed by and SM are maintained on-chip during the entire lifetime of the warp.

Therefore switching from one execution context to another has no cost.

1 SM -> 128 cores <- can execute 4 warps parallelly in one SM

How about memory latency?

lets consider DRAM latency of Maxwell architecture as 350 cycles.

T4 has 300 GB / s memory bandwidth



In [ ]:
!nvidia-smi -a -q -d CLOCK


==============NVSMI LOG==============

Timestamp                                 : Tue Apr 29 15:18:52 2025
Driver Version                            : 550.54.15
CUDA Version                              : 12.4

Attached GPUs                             : 1
GPU 00000000:00:04.0
    Clocks
        Graphics                          : 300 MHz
        SM                                : 300 MHz
        Memory                            : 405 MHz
        Video                             : 540 MHz
    Applications Clocks
        Graphics                          : 585 MHz
        Memory                            : 5001 MHz
    Default Applications Clocks
        Graphics                          : 585 MHz
        Memory                            : 5001 MHz
    Deferred Clocks
        Memory                            : N/A
    Max Clocks
        Graphics                          : 1590 MHz
        SM                                : 1590 MHz
        Memory                            : 50

5 GHz memory clock

300  / 5 = 60 Bytes / cycle.

```
<SM1>  <SM2>
  |      |
  |      |
  |------|----> <DRAM>
  |      |
  |      |
<SM3>  <SM4>

```

60 * 350 = 18400Bytes

18400 / 4 = 4600 threads

4600 / 32  = 148 warps

148 / 13  = 12 warps per SM

Categorizing CUDA applications

- Bandwidth bound applications -- arithmetic latency
- Computation bound applications -- memory latency


Occupancy is the ratio of active warps to maximum number of warps, per SM.

Occupancy  = $\frac{Active warps}{maximum warps}$

If one warp stalls execution core will be busy

maximum warps is found in documentation

active warps depend on device usage


Our kernel use 48 resiters per thread and 4096 bytes of Smem per block. And block size is 128.

Reg per warp = 48 *32 = 1536

For GTX 970 device = 65536 regs per SM

Allowed warps = 65536 / 1536 = 42.67

For GTX 970 device 98304 regs per SM

Active blocks = 98304 / 4096 = 24

active warp = 24 * 4 = 96

Actice warp does not limit by smem usage.




In [ ]:
%%writefile occupancy.cu
#include <stdio.h>
#include <cuda_runtime.h>

// Your kernel
__global__ void occupancy_test(int *results) {
    int gid = blockIdx.x * blockDim.x + threadIdx.x;
    int x1 = 1, x2 = 2, x3 = 3, x4 = 4, x5 = 5, x6 = 6, x7 = 7, x8 = 8;
    results[gid] = x1 + x2 + x3 + x4 + x5 + x6 + x7 + x8;
}

int main() {
    const int blockSize = 128;  // Number of threads per block
    const int numBlocks = 80;   // Adjust based on your GPU
    const int numThreads = blockSize * numBlocks;

    // Allocate memory on device
    int *d_results;
    cudaMalloc((void**)&d_results, numThreads * sizeof(int));

    // Launch the kernel
    occupancy_test<<<numBlocks, blockSize>>>(d_results);
    cudaDeviceSynchronize();

    // Check for errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        printf("CUDA Error: %s\n", cudaGetErrorString(err));
    }

    // Occupancy calculation
    int maxActiveBlocksPerSM = 0;
    cudaOccupancyMaxActiveBlocksPerMultiprocessor(
        &maxActiveBlocksPerSM,
        occupancy_test,
        blockSize,
        0  // dynamic shared memory
    );

    cudaDeviceProp prop;
    cudaGetDeviceProperties(&prop, 0);
    int numSMs = prop.multiProcessorCount;
    int maxThreadsPerSM = prop.maxThreadsPerMultiProcessor;

    float occupancy = (maxActiveBlocksPerSM * blockSize) / (float)maxThreadsPerSM;

    printf("Max active blocks per SM: %d\n", maxActiveBlocksPerSM);
    printf("Threads per SM: %d\n", maxActiveBlocksPerSM * blockSize);
    printf("Max threads per SM (hardware limit): %d\n", maxThreadsPerSM);
    printf("Occupancy: %.2f%%\n", occupancy * 100);

    // Free memory
    cudaFree(d_results);
    return 0;
}

Writing occupancy.cu


In [ ]:
!nvcc -o occupancy occupancy.cu
!./occupancy

occupancy.cu(41): warning #177-D: variable "numSMs" was declared but never referenced
      int numSMs = prop.multiProcessorCount;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

CUDA Error: the provided PTX was compiled with an unsupported toolchain.
Max active blocks per SM: 0
Threads per SM: 0
Max threads per SM (hardware limit): 1024
Occupancy: 0.00%


if a kernel is not bandwidth-bound or computation-bound, then increasing occupancy will not necessarily increase performance. In fact, making changes just to increase occupany can have other effects such as additional instructions, more register spills to local memory which is an off-chip memory, more divergent branches.

Keep the number of threads per bloack a multiple of warp size (32).

Keep number of blocks much greater than the number of SMs to expose sufficient parallelism to your device


## Profiling with nvprof

Profile driven optimization

Use profiling information to optimize the performance of a program in iterative manner


nvprof profile modes
- summary mode
- gpu and api trace mode
- event metrics summery mode
- event, metrics trace mode
```
nvprof[options]
    [application]
    [application-arguments]
```

In [ ]:
%%writefile hello.cu
#include <iostream>
#include <cuda_runtime.h>

__global__ void my_kernel(int *d_array) {
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    d_array[idx] = idx * idx;
}

int main() {
    int N = 1024;
    int *d_array;

    cudaMalloc(&d_array, N * sizeof(int));

    // Launch kernel
    my_kernel<<<4, 256>>>(d_array);

    cudaDeviceSynchronize();
    cudaFree(d_array);

    std::cout << "Kernel executed!" << std::endl;
    return 0;
}


Writing hello.cu


In [ ]:
!nvcc hello.cu -o hello
!nvprof ./hello


==640== NVPROF is profiling process 640, command: ./hello
Kernel executed!
==640== Profiling application: ./hello
==640== Profiling result:
No kernels were profiled.
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
      API calls:   70.23%  112.16ms         1  112.16ms  112.16ms  112.16ms  cudaMalloc
                   29.04%  46.367ms         1  46.367ms  46.367ms  46.367ms  cudaLaunchKernel
                    0.53%  853.60us         1  853.60us  853.60us  853.60us  cuDeviceGetPCIBusId
                    0.11%  171.74us       114  1.5060us     110ns  64.586us  cuDeviceGetAttribute
                    0.07%  118.91us         1  118.91us  118.91us  118.91us  cudaFree
                    0.01%  11.577us         1  11.577us  11.577us  11.577us  cuDeviceGetName
                    0.01%  9.3500us         1  9.3500us  9.3500us  9.3500us  cudaDeviceSynchronize
                    0.00%  1.4240us         3     474ns     132ns  1.0370us  cuDeviceGetCount
   

In [ ]:
!nvprof --print-gpu-trace ./hello

==847== NVPROF is profiling process 847, command: ./hello
Kernel executed!
==847== Profiling application: ./hello
==847== Profiling result:
No kernels were profiled.


In [ ]:
%%writefile heavy_kernel.cu
#include <iostream>
#include <cuda_runtime.h>

__global__ void vector_mul_and_sum(float *a, float *b, float *result, int N) {
    __shared__ float cache[256];
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    int cacheIndex = threadIdx.x;

    float temp = 0;
    while (tid < N) {
        temp += a[tid] * b[tid];
        tid += blockDim.x * gridDim.x;
    }

    cache[cacheIndex] = temp;
    __syncthreads();

    // Parallel reduction within the block
    int i = blockDim.x / 2;
    while (i != 0) {
        if (cacheIndex < i)
            cache[cacheIndex] += cache[cacheIndex + i];
        __syncthreads();
        i /= 2;
    }

    if (cacheIndex == 0)
        atomicAdd(result, cache[0]);
}

int main() {
    const int N = 1 << 20; // 1 million elements
    size_t size = N * sizeof(float);

    float *h_a = new float[N];
    float *h_b = new float[N];
    float h_result = 0;

    for (int i = 0; i < N; ++i) {
        h_a[i] = 1.0f;
        h_b[i] = 2.0f;
    }

    float *d_a, *d_b, *d_result;
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_result, sizeof(float));
    cudaMemcpy(d_a, h_a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_result, &h_result, sizeof(float), cudaMemcpyHostToDevice);

    // Launch with 256 threads per block and 256 blocks
    vector_mul_and_sum<<<256, 256>>>(d_a, d_b, d_result, N);

    cudaMemcpy(&h_result, d_result, sizeof(float), cudaMemcpyDeviceToHost);

    std::cout << "Dot product result: " << h_result << std::endl;

    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_result);
    delete[] h_a;
    delete[] h_b;

    return 0;
}


Writing heavy_kernel.cu


In [ ]:
!nvcc heavy_kernel.cu -o heavy_kernel
!nvprof ./heavy_kernel

==1542== NVPROF is profiling process 1542, command: ./heavy_kernel
Dot product result: 0
==1542== Profiling application: ./heavy_kernel
==1542== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.87%  1.5298ms         3  509.93us     640ns  772.88us  [CUDA memcpy HtoD]
                    0.13%  2.0480us         1  2.0480us  2.0480us  2.0480us  [CUDA memcpy DtoH]
      API calls:   88.49%  89.685ms         3  29.895ms  69.576us  89.543ms  cudaMalloc
                    7.82%  7.9226ms         1  7.9226ms  7.9226ms  7.9226ms  cudaLaunchKernel
                    3.03%  3.0754ms         4  768.86us  26.210us  1.9750ms  cudaMemcpy
                    0.51%  514.11us         3  171.37us  112.90us  207.44us  cudaFree
                    0.13%  126.78us       114  1.1120us     103ns  51.304us  cuDeviceGetAttribute
                    0.01%  11.627us         1  11.627us  11.627us  11.627us  cuDeviceGetName
                

## Parallel reduction as syncrhonization example

- cudaDeviceSynchronize : introduce a global synchronize point in host code.

- __syncthreads : syncrhnization with in a block

Parallel reduction : General problem of performing commutative and associative operation across vector is known as the reduction problem.

Sequential reduction:
```
int sum =0 ;
for (int i= 0 ; i < size ; i++){
  sum+= array[i]
}
```
Our approach :

- Partition the input vector into smaller chunks.
- And each chunk will be summed up separately.
- add these partial results from each chunk into a final sum.

---------------------------------------------------

Neighbored pair approach

- we are going to calculate sum of the block in iterative manner and in each iteration selected elements are paired with their neighbor from given offset

- for the first iteratino we are going to set 1 as the offset and in each iteration, this offset will be multiplied by 2.

- and number of threads which are going to do any effective work will be divide by this offset value

Code-segment

```
for (int offset = 1 ; offset < blockdim.x ; offset *=2 )
{
  if (tid % (2*offset) == 0){
    input[tid] += input[tid + offset];
  }
}
```

After each iteration we use _syncthreads() function

In [1]:
%%writefile common.h
#ifndef COMMON_H
#define COMMON_H

enum InitType { INIT_ZERO = 0, INIT_ONE = 1, INIT_RANDOM = 2 };

int reduction_cpu(int *input, const int size);
void compare_results(int gpu_result, int cpu_result);
void initialize(int *input, int size, int init_type);

#endif



Writing common.h


In [2]:
%%writefile common.cpp
#include "common.h"
#include <stdio.h>
#include <stdlib.h>
#include <time.h>

int reduction_cpu(int *input, const int size) {
    int sum = 0;
    for (int i = 0; i < size; i++) {
        sum += input[i];
    }
    return sum;
}

void compare_results(int gpu_result, int cpu_result) {
    printf("GPU RESULT : %d, CPU RESULT : %d\n", gpu_result, cpu_result);
    if (gpu_result == cpu_result) {
        printf("PASS\n");
    } else {
        printf("FAIL\n");
    }
}

void initialize(int *input, int size, int init_type) {
    srand(time(NULL));
    for (int i = 0; i < size; ++i) {
        switch (init_type) {
            case INIT_ZERO: input[i] = 0; break;
            case INIT_ONE: input[i] = 1; break;
            case INIT_RANDOM: input[i] = rand() % 100; break;
        }
    }
}



Writing common.cpp


In [3]:
%%writefile parallel_reduction.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include "common.h"

__global__ void reduction_kernel(int *input, int *temp, int size) {
    extern __shared__ int s_data[];

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    // Load data from global memory to shared memory
    s_data[tid] = (gid < size) ? input[gid] : 0;
    __syncthreads();

    // Tree-based reduction
    for (int offset = blockDim.x / 2; offset > 0; offset >>= 1) {
        if (tid < offset) {
            s_data[tid] += s_data[tid + offset];
        }
        __syncthreads();
    }

    // Write result for this block to global memory
    if (tid == 0) {
        temp[blockIdx.x] = s_data[0];
    }
}



int main() {
    printf("=== GPU Parallel Reduction ===\n");

    int size = 1024;
    int block_size = 128;
    int byte_size = size * sizeof(int);
    int grid_size = (size + block_size - 1) / block_size;

    // Host allocations
    int *h_input = (int*)malloc(byte_size);
    int *h_temp  = (int*)malloc(grid_size * sizeof(int));

    // Initialize
    initialize(h_input, size, INIT_ONE);
    int cpu_result = reduction_cpu(h_input, size);

    // Device allocations
    int *d_input, *d_temp;
    cudaMalloc((void**)&d_input, byte_size);
    cudaMalloc((void**)&d_temp, grid_size * sizeof(int));
    cudaMemcpy(d_input, h_input, byte_size, cudaMemcpyHostToDevice);

    // Launch kernel with dynamic shared memory
    reduction_kernel<<<grid_size, block_size, block_size * sizeof(int)>>>(d_input, d_temp, size);
    cudaDeviceSynchronize();

    // Copy partial sums to host
    cudaMemcpy(h_temp, d_temp, grid_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Final reduction on CPU
    int gpu_result = 0;
    for (int i = 0; i < grid_size; ++i) {
        gpu_result += h_temp[i];
    }

    compare_results(gpu_result, cpu_result);

    // Cleanup
    free(h_input);
    free(h_temp);
    cudaFree(d_input);
    cudaFree(d_temp);

    return 0;
}


Writing parallel_reduction.cu


In [4]:
!nvcc parallel_reduction.cu common.cpp -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

=== GPU Parallel Reduction ===
GPU RESULT : 1024, CPU RESULT : 1024
PASS


## Divergence in reduction algorithm

- Force neighboring threads to perform summatino

- Interleaved pairs


Forced neighboring threads approach :

In a reduction operation, this typically refers to a pattern where:

Thread i sums the value of thread i + 1 (its neighbor).

Then, i proceeds to sum with i + 2, i + 4, etc., in a loop with offsets doubling each time.

But this causes wrap divergence.

Within a warp of 32 threads:

At offset = 1, half the threads (even tids) execute the summation.

At offset = 2, one quarter of the threads (tid % 4 == 0) do the work.

At offset = 4, only tid % 8 == 0 does.

Threads in a warp take different paths due to the if condition.

This divergence forces serialization — the warp waits for all possible paths to complete, even those where some threads are idle.

Instead of using % conditions that cause divergence, you can write reduction kernels that use shared memory and stride-based access.


In [5]:
%%writefile parallel_reduction_wrap_divergence.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include "common.h"

__global__ void reduction_kernel(int *int_array, int *temp_array, int size) {
    extern __shared__ int s_data[];

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    int *i_data = int_array + blockDim.x * blockIdx.x ; //local data block pointer

    // Load data from global memory to shared memory
    s_data[tid] = (gid < size) ? int_array[gid] : 0;
    __syncthreads();

    // Tree-based reduction
    for (int offset = 1; offset <= blockDim.x / 2; offset *= 2) {
        int index = 2 * offset * tid;
        if (index < blockDim.x) {
            s_data[index] += s_data[index + offset];
        }
        __syncthreads();
    }

    // Write result for this block to global memory
    if (tid == 0) {
        temp_array[blockIdx.x] = s_data[0];
    }
}



int main() {
    printf("=== GPU Parallel Reduction ===\n");

    int size = 1024;
    int block_size = 128;
    int byte_size = size * sizeof(int);
    int grid_size = (size + block_size - 1) / block_size;

    // Host allocations
    int *h_input = (int*)malloc(byte_size);
    int *h_temp  = (int*)malloc(grid_size * sizeof(int));

    // Initialize
    initialize(h_input, size, INIT_ONE);
    int cpu_result = reduction_cpu(h_input, size);

    // Device allocations
    int *d_input, *d_temp;
    cudaMalloc((void**)&d_input, byte_size);
    cudaMalloc((void**)&d_temp, grid_size * sizeof(int));
    cudaMemcpy(d_input, h_input, byte_size, cudaMemcpyHostToDevice);

    // Launch kernel with dynamic shared memory
    reduction_kernel<<<grid_size, block_size, block_size * sizeof(int)>>>(d_input, d_temp, size);
    cudaDeviceSynchronize();

    // Copy partial sums to host
    cudaMemcpy(h_temp, d_temp, grid_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Final reduction on CPU
    int gpu_result = 0;
    for (int i = 0; i < grid_size; ++i) {
        gpu_result += h_temp[i];
    }

    compare_results(gpu_result, cpu_result);

    // Cleanup
    free(h_input);
    free(h_temp);
    cudaFree(d_input);
    cudaFree(d_temp);

    return 0;
}


Writing parallel_reduction_wrap_divergence.cu


In [6]:
!nvcc parallel_reduction_wrap_divergence.cu common.cpp -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

parallel_reduction_wrap_divergence.cu(12): warning #177-D: variable "i_data" was declared but never referenced
      int *i_data = int_array + blockDim.x * blockIdx.x ;
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

=== GPU Parallel Reduction ===
GPU RESULT : 1024, CPU RESULT : 1024
PASS


Interleaved pair :

In [17]:
%%writefile parallel_reduction_wrap_divergence_interleaved.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include "common.h"

__global__ void reduction_kernel(int *int_array, int *temp_array, int size) {
    extern __shared__ int s_data[];

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    int *i_data = int_array + blockDim.x * blockIdx.x ; //local data block pointer

    // Load data from global memory to shared memory
    s_data[tid] = (gid < size) ? int_array[gid] : 0;
    __syncthreads();

    // Tree-based reduction
    for (int offset = blockDim.x / 2; offset > 0 ; offset >>=1) {

        if (tid < offset) {
            s_data[tid] += s_data[tid + offset];
        }
        __syncthreads();
    }

    // Write result for this block to global memory
    if (tid == 0) {
        temp_array[blockIdx.x] = s_data[0];
    }
}



int main() {
    printf("=== GPU Parallel Reduction ===\n");

    int size = 1024;
    int block_size = 128;
    int byte_size = size * sizeof(int);
    int grid_size = (size + block_size - 1) / block_size;

    // Host allocations
    int *h_input = (int*)malloc(byte_size);
    int *h_temp  = (int*)malloc(grid_size * sizeof(int));

    // Initialize
    initialize(h_input, size, INIT_ONE);
    int cpu_result = reduction_cpu(h_input, size);

    // Device allocations
    int *d_input, *d_temp;
    cudaMalloc((void**)&d_input, byte_size);
    cudaMalloc((void**)&d_temp, grid_size * sizeof(int));
    cudaMemcpy(d_input, h_input, byte_size, cudaMemcpyHostToDevice);

    // Launch kernel with dynamic shared memory
    reduction_kernel<<<grid_size, block_size, block_size * sizeof(int)>>>(d_input, d_temp, size);
    cudaDeviceSynchronize();

    // Copy partial sums to host
    cudaMemcpy(h_temp, d_temp, grid_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Final reduction on CPU
    int gpu_result = 0;
    for (int i = 0; i < grid_size; ++i) {
        gpu_result += h_temp[i];
    }

    compare_results(gpu_result, cpu_result);

    // Cleanup
    free(h_input);
    free(h_temp);
    cudaFree(d_input);
    cudaFree(d_temp);

    return 0;
}


Overwriting parallel_reduction_wrap_divergence_interleaved.cu


In [18]:
!nvcc parallel_reduction_wrap_divergence_interleaved.cu common.cpp -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

parallel_reduction_wrap_divergence_interleaved.cu(12): warning #177-D: variable "i_data" was declared but never referenced
      int *i_data = int_array + blockDim.x * blockIdx.x ;
           ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

=== GPU Parallel Reduction ===
GPU RESULT : 1024, CPU RESULT : 1024
PASS


## Loop unrolling

- In loop unrolling, rather than writing the body of a loop once and using a loop to execute it repeatedly, the body is written in code multiple times.

- The number of copies made of the loop body is called loop unrolling factor.

Threads blocks unrolling

Data blocks
```
A -> thread block
B -> thread block
C -> thread block
D -> thread block

A \
    Threadblock
B /
C \
    ThreadBlock
D /
```

In [15]:
%%writefile parallel_reduction_wrap_unrolling.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include "common.h"

__global__ void reduction_unrolling_blocks2(int *int_array, int *temp_array, int size) {
    extern __shared__ int s_data[];

    int tid = threadIdx.x;
    int BLOCK_OFFSET = blockIdx.x * blockDim.x * 2;
    int index = BLOCK_OFFSET + tid;

    // Load two elements per thread from global memory
    int sum = 0;
    if (index < size) {
        sum = int_array[index];
        if (index + blockDim.x < size) {
            sum += int_array[index + blockDim.x];
        }
    }

    s_data[tid] = sum;
    __syncthreads();

    // Standard reduction in shared memory
    for (int offset = blockDim.x / 2; offset > 0; offset >>= 1) {
        if (tid < offset) {
            s_data[tid] += s_data[tid + offset];
        }
        __syncthreads();
    }

    if (tid == 0) {
        temp_array[blockIdx.x] = s_data[0];
    }
}


__global__ void reduction_unrolling_blocks4(int *int_array, int *temp_array, int size) {
    extern __shared__ int s_data[];

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    int BLOCK_OFFSET = blockIdx.x * blockDim.x * 2;
    int index = BLOCK_OFFSET + tid;

    // Load data from global memory to shared memory
    s_data[tid] = (gid < size) ? int_array[gid] : 0;
    __syncthreads();

    if ((index + 3 * blockDim.x) < size){
      s_data[index] += s_data[index + blockDim.x];
      int a1 = s_data[index];
      int a2 = s_data[index + blockDim.x];
      int a3 = s_data[index + 2 * blockDim.x];
      int a4 = s_data[index + 3 * blockDim.x];

      s_data[index] = a1+ a2 + a3+ a4;
    }
    __syncthreads();
    // Tree-based reduction
    for (int offset = blockDim.x / 2; offset > 0 ; offset = offset / 2) {

        if (tid < offset) {
            s_data[tid] += s_data[tid + offset];
        }
        __syncthreads();
    }

    // Write result for this block to global memory
    if (tid == 0) {
        temp_array[blockIdx.x] = s_data[0];
    }
}

int main() {
    printf("=== GPU Parallel Reduction ===\n");

    int size = 1024;
    int block_size = 128;
    int byte_size = size * sizeof(int);
    // int grid_size = (size + block_size - 1) / block_size; // for one block kernel
    int grid_size = (size + block_size * 2 - 1) / (block_size * 2); // for 2 block kernel

    // Host allocations
    int *h_input = (int*)malloc(byte_size);
    int *h_temp  = (int*)malloc(grid_size * sizeof(int));

    // Initialize
    initialize(h_input, size, INIT_ONE);
    int cpu_result = reduction_cpu(h_input, size);

    // Device allocations
    int *d_input, *d_temp;
    cudaMalloc((void**)&d_input, byte_size);
    cudaMalloc((void**)&d_temp, grid_size * sizeof(int));
    cudaMemcpy(d_input, h_input, byte_size, cudaMemcpyHostToDevice);

    // Launch kernel with dynamic shared memory
    reduction_unrolling_blocks2<<<grid_size, block_size, block_size * sizeof(int)>>>(d_input, d_temp, size);
    cudaDeviceSynchronize();

    // Copy partial sums to host
    cudaMemcpy(h_temp, d_temp, grid_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Final reduction on CPU
    int gpu_result = 0;
    for (int i = 0; i < grid_size; ++i) {
        gpu_result += h_temp[i];
    }

    compare_results(gpu_result, cpu_result);

    // Cleanup
    free(h_input);
    free(h_temp);
    cudaFree(d_input);
    cudaFree(d_temp);

    return 0;
}


Overwriting parallel_reduction_wrap_unrolling.cu


In [16]:
!nvcc parallel_reduction_wrap_unrolling.cu common.cpp -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

=== GPU Parallel Reduction ===
GPU RESULT : 1024, CPU RESULT : 1024
PASS


## WRAP UNROLLLING

Warp unrolling is a GPU optimization technique where instead of each thread processing a single element, each thread processes multiple elements (usually from global memory) within a warp (32 threads). This increases memory throughput and reduces the number of thread launches.

Why is it useful :
> Memory coalescing: Modern GPUs fetch global memory in 32-, 64-, or 128-byte chunks. Unrolling helps align memory accesses efficiently.

> Reduced instruction overhead: Fewer iterations, fewer branches.

> Higher occupancy and arithmetic intensity.


We'll assume:

Array size: 8 elements → int_array = [1, 2, 3, 4, 5, 6, 7, 8]

blockDim.x = 8

gridDim.x = 1 (just one block)

So we'll launch 8 threads → tid from 0 to 7

For simplicity, assume each thread reads one value from int_array

Focus is on how the reduction is done using tree reduction + warp unrolling


stopping at 64 prevents warp divergence by:
Ensuring the loop only runs when there's no mixed-path execution within a warp

Switching to warp-synchronous code (manual unrolling or shfl) for the last 64 elements

Avoiding __syncthreads() in the warp phase — which is both unnecessary and unavailable






In [20]:
%%writefile parallel_reduction_wrap_unrolling.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include "common.h"

__global__ void reduction_kernel_wrap_unrolling(int *int_array, int *temp_array, int size) {

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    int *i_data = int_array + blockDim.x * blockIdx.x ; //local data block pointer


    // Tree-based reduction
    for (int offset = blockDim.x / 2; offset >=64  ; offset  = offset / 2) {

        if (tid < offset) {
            i_data[tid] += i_data[tid + offset];
        }
        __syncthreads();
    }
    if (tid < 32){
      volatile int *vsmem = i_data; // happen directly without any caches.
      vsmem[tid] += vsmem[tid + 32];
      vsmem[tid] += vsmem[tid + 16];
      vsmem[tid] += vsmem[tid + 8];
      vsmem[tid] += vsmem[tid + 4];
      vsmem[tid] += vsmem[tid +2];
      vsmem[tid] += vsmem[tid +1];
    }
    // Write result for this block to global memory
    if (tid == 0) {
        temp_array[blockIdx.x] = i_data[0];
    }
}



int main() {
    printf("=== GPU Parallel Reduction ===\n");

    int size = 1024;
    int block_size = 128;
    int byte_size = size * sizeof(int);
    int grid_size = (size + block_size - 1) / block_size;

    // Host allocations
    int *h_input = (int*)malloc(byte_size);
    int *h_temp  = (int*)malloc(grid_size * sizeof(int));

    // Initialize
    initialize(h_input, size, INIT_ONE);
    int cpu_result = reduction_cpu(h_input, size);

    // Device allocations
    int *d_input, *d_temp;
    cudaMalloc((void**)&d_input, byte_size);
    cudaMalloc((void**)&d_temp, grid_size * sizeof(int));
    cudaMemcpy(d_input, h_input, byte_size, cudaMemcpyHostToDevice);

    // Launch kernel with dynamic shared memory
    reduction_kernel_wrap_unrolling<<<grid_size, block_size, block_size * sizeof(int)>>>(d_input, d_temp, size);
    cudaDeviceSynchronize();

    // Copy partial sums to host
    cudaMemcpy(h_temp, d_temp, grid_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Final reduction on CPU
    int gpu_result = 0;
    for (int i = 0; i < grid_size; ++i) {
        gpu_result += h_temp[i];
    }

    compare_results(gpu_result, cpu_result);

    // Cleanup
    free(h_input);
    free(h_temp);
    cudaFree(d_input);
    cudaFree(d_temp);

    return 0;
}


Overwriting parallel_reduction_wrap_unrolling.cu


In [21]:
!nvcc parallel_reduction_wrap_unrolling.cu common.cpp -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

parallel_reduction_wrap_unrolling.cu(10): warning #177-D: variable "gid" was declared but never referenced
      int gid = blockIdx.x * blockDim.x + tid;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

=== GPU Parallel Reduction ===
GPU RESULT : 1024, CPU RESULT : 1024
PASS


## Reduction with complete unrolling

Complete unrolling means you eliminate all loops in the final stages of reduction (especially within a warp) and manually write out each step.

1st iteration : offset = 512 : first 512 elements will have sum of next 512
.....
in last iteration till 64 offset we get sum of all elements in the first 64 elements


In [23]:
%%writefile reduction_kernel_complete_unrolling.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include "common.h"

__global__ void reduction_kernel_complete_unrolling(int *int_array, int *temp_array, int size) {

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    int *i_data = int_array + blockDim.x * blockIdx.x ; //local data block pointer


    if(blockDim.x == 1024 && tid < 512)
      i_data[tid] += i_data[tid + 512];
    __syncthreads();

    if(blockDim.x == 512 && tid < 256)
      i_data[tid] += i_data[tid +256];
    __syncthreads();

    if(blockDim.x == 256 && tid < 128)
      i_data[tid] += i_data[tid +128];
    __syncthreads();

    if(blockDim.x == 128 && tid < 64)
      i_data[tid] += i_data[tid +64];
    __syncthreads();

    if (tid < 32){
      volatile int *vsmem = i_data; // happen directly without any caches.
      vsmem[tid] += vsmem[tid + 32];
      vsmem[tid] += vsmem[tid + 16];
      vsmem[tid] += vsmem[tid + 8];
      vsmem[tid] += vsmem[tid + 4];
      vsmem[tid] += vsmem[tid +2];
      vsmem[tid] += vsmem[tid +1];
    }
    // Write result for this block to global memory
    if (tid == 0) {
        temp_array[blockIdx.x] = i_data[0];
    }
}



int main() {
    printf("=== GPU Parallel Reduction ===\n");

    int size = 1024;
    int block_size = 128;
    int byte_size = size * sizeof(int);
    int grid_size = (size + block_size - 1) / block_size;

    // Host allocations
    int *h_input = (int*)malloc(byte_size);
    int *h_temp  = (int*)malloc(grid_size * sizeof(int));

    // Initialize
    initialize(h_input, size, INIT_ONE);
    int cpu_result = reduction_cpu(h_input, size);

    // Device allocations
    int *d_input, *d_temp;
    cudaMalloc((void**)&d_input, byte_size);
    cudaMalloc((void**)&d_temp, grid_size * sizeof(int));
    cudaMemcpy(d_input, h_input, byte_size, cudaMemcpyHostToDevice);

    // Launch kernel with dynamic shared memory
    reduction_kernel_complete_unrolling<<<grid_size, block_size, block_size * sizeof(int)>>>(d_input, d_temp, size);
    cudaDeviceSynchronize();

    // Copy partial sums to host
    cudaMemcpy(h_temp, d_temp, grid_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Final reduction on CPU
    int gpu_result = 0;
    for (int i = 0; i < grid_size; ++i) {
        gpu_result += h_temp[i];
    }

    compare_results(gpu_result, cpu_result);

    // Cleanup
    free(h_input);
    free(h_temp);
    cudaFree(d_input);
    cudaFree(d_temp);

    return 0;
}


Overwriting reduction_kernel_complete_unrolling.cu


In [24]:
!nvcc reduction_kernel_complete_unrolling.cu common.cpp -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

reduction_kernel_complete_unrolling.cu(9): warning #177-D: variable "gid" was declared but never referenced
      int gid = blockIdx.x * blockDim.x + tid;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

=== GPU Parallel Reduction ===
GPU RESULT : 1024, CPU RESULT : 1024
PASS


Parallel reduction algorithms :
- naive neighbored pairs approach
- interleaved pair approach
- data block unrolling
- warp unrolling
- completely unrolling


## Dynamic parallelism


Dynamic Parallelism allows a CUDA kernel to launch other kernels directly from the GPU (without going back to the CPU).

This is useful for problems where:

- The amount of work is data-dependent

- You need recursion or nested computation

- You want to offload CPU kernel launch overhead

```
__global__ void child_kernel(int *data) {
    int idx = threadIdx.x;
    data[idx] += 1;
}

__global__ void parent_kernel(int *data) {
    if (threadIdx.x == 0) {
        // GPU launching child kernel
        child_kernel<<<1, 10>>>(data);
        cudaDeviceSynchronize(); // wait for child to finish
    }
}

```

- Parent and child grids share the same global and constant memory storage but have distinct local and shared memory

- There are two points in the execution of a child grid when its view of memory is fully consistent with the parent thread: at the start of a child grid, and when the child grid completes.

- Shared and local memory are private to a thread block or thread, respectively and are not visible or coherent between parent and child.

- Parents kernel will be launched from host with one thread block
- in each grid, first thread block in the thread block has to launch the child grid.




In [29]:
%%writefile dynamic_parallelism.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <time.h>
#include "common.h"

__global__ void dynamic_parallelism_check(int size, int depth) {

    printf("Depth : %d - tid : %d\n", depth, threadIdx.x);
    if (size == 1){
      return;
    }
    if (threadIdx.x == 0){
      dynamic_parallelism_check<<<1, size / 2>>>(size/2, depth +1);
    }
}



int main(int argc, char** argv) {

  dynamic_parallelism_check<<< 1, 16 >>> (16, 0);
  cudaDeviceSynchronize();
  cudaDeviceReset();

  return 0;
}


Overwriting dynamic_parallelism.cu


In [30]:
!nvcc -arch=sm_50 -rdc=true dynamic_parallelism.cu -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

Depth : 0 - tid : 0
Depth : 0 - tid : 1
Depth : 0 - tid : 2
Depth : 0 - tid : 3
Depth : 0 - tid : 4
Depth : 0 - tid : 5
Depth : 0 - tid : 6
Depth : 0 - tid : 7
Depth : 0 - tid : 8
Depth : 0 - tid : 9
Depth : 0 - tid : 10
Depth : 0 - tid : 11
Depth : 0 - tid : 12
Depth : 0 - tid : 13
Depth : 0 - tid : 14
Depth : 0 - tid : 15
Depth : 1 - tid : 0
Depth : 1 - tid : 1
Depth : 1 - tid : 2
Depth : 1 - tid : 3
Depth : 1 - tid : 4
Depth : 1 - tid : 5
Depth : 1 - tid : 6
Depth : 1 - tid : 7
Depth : 2 - tid : 0
Depth : 2 - tid : 1
Depth : 2 - tid : 2
Depth : 2 - tid : 3
Depth : 3 - tid : 0
Depth : 3 - tid : 1
Depth : 4 - tid : 0


In [34]:
!nvprof ./reduction

==19192== NVPROF is profiling process 19192, command: ./reduction
Depth : 0 - tid : 0
Depth : 0 - tid : 1
Depth : 0 - tid : 2
Depth : 0 - tid : 3
Depth : 0 - tid : 4
Depth : 0 - tid : 5
Depth : 0 - tid : 6
Depth : 0 - tid : 7
Depth : 0 - tid : 8
Depth : 0 - tid : 9
Depth : 0 - tid : 10
Depth : 0 - tid : 11
Depth : 0 - tid : 12
Depth : 0 - tid : 13
Depth : 0 - tid : 14
Depth : 0 - tid : 15
Depth : 1 - tid : 0
Depth : 1 - tid : 1
Depth : 1 - tid : 2
Depth : 1 - tid : 3
Depth : 1 - tid : 4
Depth : 1 - tid : 5
Depth : 1 - tid : 6
Depth : 1 - tid : 7
Depth : 2 - tid : 0
Depth : 2 - tid : 1
Depth : 2 - tid : 2
Depth : 2 - tid : 3
Depth : 3 - tid : 0
Depth : 3 - tid : 1
Depth : 4 - tid : 0
==19192== Profiling application: ./reduction
==19192== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  124.83us         1  124.83us  124.83us  124.83us  dynamic_parallelism_check(int, int)
      API calls:   73.88%  93.752ms     

In [70]:
%%writefile assingment_1.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <time.h>
#include "common.h"

__global__ void dynamic_parallelism_check(int size, int depth, int parent_id) {
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    printf("Parent : %d - Depth : %d - tid : %d - gid : %d blockID: %d\n", parent_id ,depth, tid, gid, blockIdx.x);

    if (size == 1){
      return;
    }

    if (threadIdx.x == 0 && blockIdx.x == 0){
      dynamic_parallelism_check<<<1, size / 2>>>(size/2, depth +1, blockIdx.x);
    }
}



int main(int argc, char** argv) {

  dynamic_parallelism_check<<< 2, 8 >>> (8, 0, -1);
  cudaDeviceSynchronize();
  cudaDeviceReset();

  return 0;
}


Overwriting assingment_1.cu


In [71]:
!nvcc -arch=sm_50 -rdc=true assingment_1.cu -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

Parent : -1 - Depth : 0 - tid : 0 - gid : 8 blockID: 1
Parent : -1 - Depth : 0 - tid : 1 - gid : 9 blockID: 1
Parent : -1 - Depth : 0 - tid : 2 - gid : 10 blockID: 1
Parent : -1 - Depth : 0 - tid : 3 - gid : 11 blockID: 1
Parent : -1 - Depth : 0 - tid : 4 - gid : 12 blockID: 1
Parent : -1 - Depth : 0 - tid : 5 - gid : 13 blockID: 1
Parent : -1 - Depth : 0 - tid : 6 - gid : 14 blockID: 1
Parent : -1 - Depth : 0 - tid : 7 - gid : 15 blockID: 1
Parent : -1 - Depth : 0 - tid : 0 - gid : 0 blockID: 0
Parent : -1 - Depth : 0 - tid : 1 - gid : 1 blockID: 0
Parent : -1 - Depth : 0 - tid : 2 - gid : 2 blockID: 0
Parent : -1 - Depth : 0 - tid : 3 - gid : 3 blockID: 0
Parent : -1 - Depth : 0 - tid : 4 - gid : 4 blockID: 0
Parent : -1 - Depth : 0 - tid : 5 - gid : 5 blockID: 0
Parent : -1 - Depth : 0 - tid : 6 - gid : 6 blockID: 0
Parent : -1 - Depth : 0 - tid : 7 - gid : 7 blockID: 0
Parent : 0 - Depth : 1 - tid : 0 - gid : 0 blockID: 0
Parent : 0 - Depth : 1 - tid : 1 - gid : 1 blockID: 0
Parent

In [68]:
%%writefile assingment_2.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <time.h>
#include "common.h"

__global__ void dynamic_parallelism_check(int size, int depth, int parent_id) {
    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    printf("Parent : %d - Depth : %d - tid : %d - gid : %d blockID: %d\n", parent_id ,depth, tid, gid, blockIdx.x);

    if (size == 1){
      return;
    }

    if (threadIdx.x == 0){
      dynamic_parallelism_check<<<1, size / 2>>>(size/2, depth +1, blockIdx.x);
    }
}



int main(int argc, char** argv) {

  dynamic_parallelism_check<<< 2, 8 >>> (8, 0, -1);
  cudaDeviceSynchronize();
  cudaDeviceReset();

  return 0;
}


Overwriting assingment_2.cu


In [69]:
!nvcc -arch=sm_50 -rdc=true assingment_2.cu -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

Parent : -1 - Depth : 0 - tid : 0 - gid : 8 blockID: 1
Parent : -1 - Depth : 0 - tid : 1 - gid : 9 blockID: 1
Parent : -1 - Depth : 0 - tid : 2 - gid : 10 blockID: 1
Parent : -1 - Depth : 0 - tid : 3 - gid : 11 blockID: 1
Parent : -1 - Depth : 0 - tid : 4 - gid : 12 blockID: 1
Parent : -1 - Depth : 0 - tid : 5 - gid : 13 blockID: 1
Parent : -1 - Depth : 0 - tid : 6 - gid : 14 blockID: 1
Parent : -1 - Depth : 0 - tid : 7 - gid : 15 blockID: 1
Parent : -1 - Depth : 0 - tid : 0 - gid : 0 blockID: 0
Parent : -1 - Depth : 0 - tid : 1 - gid : 1 blockID: 0
Parent : -1 - Depth : 0 - tid : 2 - gid : 2 blockID: 0
Parent : -1 - Depth : 0 - tid : 3 - gid : 3 blockID: 0
Parent : -1 - Depth : 0 - tid : 4 - gid : 4 blockID: 0
Parent : -1 - Depth : 0 - tid : 5 - gid : 5 blockID: 0
Parent : -1 - Depth : 0 - tid : 6 - gid : 6 blockID: 0
Parent : -1 - Depth : 0 - tid : 7 - gid : 7 blockID: 0
Parent : 1 - Depth : 1 - tid : 0 - gid : 0 blockID: 0
Parent : 1 - Depth : 1 - tid : 1 - gid : 1 blockID: 0
Parent

## Reduction with dynamic parallelism

In [98]:
%%writefile reduction_dynamic_programming.cu
#include <stdio.h>
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include "common.h"

__global__ void gpuRecursiveReduce(int *input, int *output, int size) {
    extern __shared__ int sdata[];

    int tid = threadIdx.x;
    int gid = blockIdx.x * blockDim.x + tid;

    // Load global data into shared memory
    sdata[tid] = (gid < size) ? input[gid] : 0;
    __syncthreads();

    // Interleaved reduction
    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (tid < stride) {
            sdata[tid] += sdata[tid + stride];
        }
        __syncthreads();
    }

    // Store per-block result
    if (tid == 0) {
        output[blockIdx.x] = sdata[0];
    }

    // Recursively reduce further
    if (tid == 0 && blockIdx.x == 0) {
        int num_blocks = (size + blockDim.x - 1) / blockDim.x;
        if (num_blocks > 1) {
            int new_blocks = (num_blocks + blockDim.x - 1) / blockDim.x;
            gpuRecursiveReduce<<<new_blocks, blockDim.x, blockDim.x * sizeof(int)>>>(output, output, num_blocks);
        }
    }
}

int main() {
    const int size = 1024;
    const int block_size = 128;
    const int bytes = size * sizeof(int);

    // Allocate and initialize host memory
    int *h_input = (int *)malloc(bytes);
    for (int i = 0; i < size; i++) h_input[i] = 1;
    int cpu_result = reduction_cpu(h_input, size);  // should return 1024

    // Allocate device memory
    int *d_input, *d_output;
    cudaMalloc(&d_input, bytes);
    cudaMalloc(&d_output, bytes);  // reuse same buffer for recursive output

    cudaMemcpy(d_input, h_input, bytes, cudaMemcpyHostToDevice);

    // Allow recursive launches
    cudaDeviceSetLimit(cudaLimitDevRuntimeSyncDepth, 8);

    int grid_size = (size + block_size - 1) / block_size;

    // Launch recursive reduction kernel
    gpuRecursiveReduce<<<grid_size, block_size, block_size * sizeof(int)>>>(d_input, d_output, size);
    cudaDeviceSynchronize();

    // Read final result from d_output[0]
    int gpu_result = 0;
    cudaMemcpy(&gpu_result, d_output, sizeof(int), cudaMemcpyDeviceToHost);

    //printf("GPU RESULT : %d, CPU RESULT : %d\n", gpu_result, cpu_result);
    compare_results(gpu_result, cpu_result);

    cudaFree(d_input);
    cudaFree(d_output);
    free(h_input);

    return 0;
}



Overwriting reduction_dynamic_programming.cu


In [99]:
!nvcc -arch=sm_50 -rdc=true reduction_dynamic_programming.cu common.cpp -o reduction -gencode arch=compute_75,code=sm_75
!./reduction

GPU RESULT : 1024, CPU RESULT : 1024
PASS


In [100]:
!nvprof ./reduction

==42701== NVPROF is profiling process 42701, command: ./reduction
GPU RESULT : 1024, CPU RESULT : 1024
PASS
==42701== Profiling application: ./reduction
==42701== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   91.93%  40.480us         1  40.480us  40.480us  40.480us  gpuRecursiveReduce(int*, int*, int)
                    4.80%  2.1120us         1  2.1120us  2.1120us  2.1120us  [CUDA memcpy DtoH]
                    3.27%  1.4400us         1  1.4400us  1.4400us  1.4400us  [CUDA memcpy HtoD]
      API calls:   91.42%  116.53ms         2  58.264ms  6.3060us  116.52ms  cudaMalloc
                    8.17%  10.413ms         1  10.413ms  10.413ms  10.413ms  cudaLaunchKernel
                    0.18%  231.23us       114  2.0280us     160ns  81.332us  cuDeviceGetAttribute
                    0.11%  134.08us         2  67.041us  12.425us  121.66us  cudaFree
                    0.05%  66.686us         1  66.686us  66.686us 

## Summary

- Warp execution
- Resouuce partition and latency hiding
- Optimizing a cuda program based on cuda execution model
- every cuda device is going to have multiple SM and each SM is going to have hundreds of cores
- Thread block is schedule to a single SM
- warp is basic unit of exeuction in a cuda program
- if the sets of threads exeucte different instructino than other part of the warp, then warp divergence occurs.
- warp divergence will hinder the perfromance of a cuda
- resoucre partitioning in a cuda program
- shared memory is a memory which shared by all the threads.
- latency of an arithmentic and memory instructions
- occupancy is a measurement of number of resident active thread blocks or wraps
- syncrhonization between threads with in thread block using __syncthread() function
- parallel reduction algorithm
-- navieneighbored pairs approach
- interlaeved pair apporach
- data block unrolling
- warp unrolling
- complete unrolling

- With dynamic parallelism we can launch cuda kernel form another kernel